<a href="https://colab.research.google.com/github/tathianamb/pdf_assistant/blob/gemini_v3/chatpdf_gemini_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U --quiet langchain-google-genai langchain faiss-cpu pypdf sentence-transformers PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 681.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
%pip install -U --quiet scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import ScaNN
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from google.colab import userdata
import google.generativeai as genai


api_key = userdata.get('GOOGLE_API_KEY')
if not api_key:
    raise ValueError("Missing GOOGLE_API_KEY environment variable")

genai.configure(api_key=api_key)

os.environ["GOOGLE_API_KEY"] = api_key



In [ ]:
class pdfQA:

  def __init__(self, model_path="models/embedding-001"):
    self.embeddings = GoogleGenerativeAIEmbeddings(model=model_path)
    self.chain = self._get_conversational_chain()


  def _get_pdf_text(self, pdf_docs):
      """Extracts text from a list of PDF documents."""
      text = ""
      for pdf in pdf_docs:
          pdf_reader = PdfReader(pdf)
          for page in pdf_reader.pages:
              text += page.extract_text()
      return text


  def _get_text_chunks(self, text):
      """Splits text into chunks for embedding."""
      text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
      chunks = text_splitter.split_text(text)
      return chunks


  def _create_vector_store(self, text_chunks, pdf_filename):
      """Creates a SCANN vector store from text chunks and embeddings."""
      embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
      db = ScaNN.from_texts(text_chunks, embeddings)
      vector_store_path = os.path.join("scan_vs", pdf_filename)
      db.save_local(vector_store_path)


  def _get_conversational_chain(self):
      """Defines the question-answering chain using a prompt template and model."""
      prompt_template = """
      Answer the question with full context details.\n\n
      Context:\n {context}?\n
      Question: \n{question}\n

      Answer:
      """

      model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

      prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
      chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

      return chain


  def _answer_user_question(self, user_question, pdf_filename):
      """Try to answers the user's question using the conversational chain and embeddings."""
      embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

      vector_store_path = os.path.join("scan_vs", pdf_filename)  # Construct the full path

      new_db = ScaNN.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)

      docs = new_db.similarity_search(user_question)

      chain = self.chain

      response = chain(
          {"input_documents": docs, "question": user_question}, return_only_outputs=True
      )

      return response["output_text"]


  def _vector_store_exists(self, pdf_filename):
      """Checks if a vector store exists for a given PDF filename."""
      vector_store_path = os.path.join("scan_vs", pdf_filename)  # Construct the full path
      return os.path.exists(vector_store_path)


  def main(self):
      """Handles user interaction, PDF processing, and question answering."""

      print("Chat with PDF using Gemini")

      try:
          existing_vector_stores = os.listdir("scan_vs")
          print("Choose a vector store:")
          for idx, vector_store in enumerate(existing_vector_stores):
            print(f"{idx + 1}. {vector_store}")

          selected = input("Enter the number corresponding to the vector store you want or 'upload' to a new file\n")

      except FileNotFoundError:
          os.makedirs("scan_vs")
          existing_vector_stores = []
          selected = input("Write 'upload' to a new file\n")

      if selected == 'upload':

          pdf_docs = input("\nEnter paths to your PDF files separated by commas (,) or 'upload' to upload from local machine\n")

          pdf_docs = pdf_docs.split(",")

          for pdf in pdf_docs:
              pdf_filename = os.path.splitext(os.path.basename(pdf))[0]
              if not self._vector_store_exists(pdf_filename):
                  raw_text = self._get_pdf_text([pdf])
                  text_chunks = self._get_text_chunks(raw_text)
                  pdf_filename = os.path.splitext(os.path.basename(pdf))[0]
                  self._create_vector_store(text_chunks, pdf_filename)
                  print(f"\nVector store created for {pdf_filename}.")
              else:
                  print(f"\nVector store already exists for {pdf_filename}.")

      else:
        selected_index = int(selected) - 1

        selected_vector_store = existing_vector_stores[selected_index]

        pdf_filename = selected_vector_store.replace("_scann_index", "")

        print(f"\nSelected vector store: {selected_vector_store}")

      print("Ask a Question from the PDF Files (or 'quit')\n")
      while True:
          user_question = input("User: ")
          if user_question.lower() == 'quit':
              break
          answer = self._answer_user_question(user_question, pdf_filename)
          # You can optionally display the answer here (if not printed earlier)
          print(f"\nGemini: \n{answer}\n\n")


In [ ]:
if __name__ == "__main__":
    qa_system = pdfQA()
    qa_system.main()

Chat with PDF using Gemini
